In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '/Users/mymac/Google Drive/My Drive/Forex_Robot/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'Oanda_Aud_Usd_M5_2021-2022.csv')
# df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2021-2022.csv')
# df = pd.read_csv(file_path + 'Oanda_Usd_Cad_M5_2021-2022.csv')
# df = pd.read_csv(file_path + 'Oanda_Gbp_Chf_M5_2021-2022.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2021-04-01 06:00:00   0.75364   0.75369  0.75324    0.75324   0.75379   
1 2021-04-01 06:05:00   0.75329   0.75362  0.75315    0.75330   0.75342   
2 2021-04-01 06:10:00   0.75326   0.75363  0.75326    0.75360   0.75340   
3 2021-04-01 06:15:00   0.75364   0.75426  0.75364    0.75426   0.75375   
4 2021-04-01 06:20:00   0.75423   0.75495  0.75407    0.75469   0.75436   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  \
0   0.75382  0.75339    0.75339   0.75372   0.75376  0.75332    0.75332   
1   0.75375  0.75327    0.75343   0.75336   0.75368  0.75321    0.75336   
2   0.75377  0.75340    0.75372   0.75333   0.75370  0.75333    0.75366   
3   0.75440  0.75375    0.75440   0.75370   0.75433  0.75370    0.75433   
4   0.75509  0.75421    0.75483   0.75430   0.75502  0.75414    0.75476   

   Volume  
0      63  
1      62  
2      48  
3      46  
4      82  
--------------------------

In [5]:
def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(barsdata, lookback=14):
    high_low = barsdata['Mid_High'] - barsdata['Mid_Low']
    high_close = np.abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    low_close = np.abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback


def rsi(barsdata, periods=14):
    close_delta = barsdata['Mid_Close'].diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def stoch_rsi(data, k_window=3, d_window=3, window=14):
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()

    stoch = ((data - min_val) / (max_val - min_val)) * 100

    slow_k = stoch.rolling(window=k_window, center=False).mean()

    slow_d = slow_k.rolling(window=d_window, center=False).mean()

    return slow_k, slow_d

def n_macd(macd, macdsignal, lookback=50):
    n_macd = 2 * (((macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min()))) - 1
    n_macdsignal = 2 * (((macdsignal - macdsignal.rolling(lookback).min()) / (macdsignal.rolling(lookback).max() - macdsignal.rolling(lookback).min()))) - 1

    return n_macd, n_macdsignal

def chop(df, lookback=14):
    atr1 = atr(df, lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

def bar_lengths(bar_lens, window=36):
    return bar_lens.rolling(window=window).mean(), bar_lens.rolling(window=window).std()

def sar_lengths(opens, sars, window=36):
    diffs = abs(opens - sars.shift(1))

    return diffs.rolling(window=window).mean(), diffs.rolling(window=window).std()

def bollinger_bands(closes, window=20):
    sma = closes.rolling(window=window).mean()
    std = closes.rolling(window=window).std()
    bollinger_up = sma + std * 2
    bollinger_down = sma - std * 2

    return bollinger_up, bollinger_down

def keltner_channels(closes, atr_10, multiplier=2):
    ema = pd.Series.ewm(closes, span=20).mean()
    keltner_up = ema + multiplier * atr_10
    keltner_down = ema - multiplier * atr_10

    return keltner_up, keltner_down

def range_high_lows(df, i, n_histograms=6):
    green, red = False, False
    curr_color, curr_streak = None, 0
    high, low = -np.inf, np.inf

    while i >= 0:
        hist = df.loc[df.index[i], 'macdhist']
        curr_high, curr_low = df.loc[df.index[i], ['Mid_High', 'Mid_Low']]
        new_color = 'green' if hist > 0 else 'red'

        if curr_color != new_color:
            curr_color = new_color
            curr_streak = 0

            if green and red:
                break

        curr_streak += 1

        if curr_streak >= n_histograms:
            high = max(high, curr_high)
            low = min(low, curr_low)

            if curr_color == 'red':
                red = True

            else:
                green = True

        i -= 1

    return high, low if (high > -np.inf and low < np.inf) else np.nan, np.nan


In [6]:
# Add technical indicators (for additional features)
df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()
df['ema50'] = pd.Series.ewm(df['Mid_Close'], span=50).mean()
df['ema25'] = pd.Series.ewm(df['Mid_Close'], span=25).mean()

df['atr'] = atr(df)
df['atr_sma'] = df['atr'].rolling(20).mean()
df['rsi'] = rsi(df, periods=18)
df['rsi_sma'] = df['rsi'].rolling(36).mean()
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
df['macdhist'] = df['macd'] - df['macdsignal']
df['n_macd'], df['n_macdsignal'] = n_macd(df['macd'], df['macdsignal'])
df['slowk'], df['slowd'] = stoch(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['slowk_rsi'], df['slowd_rsi'] = stoch_rsi(df['rsi'])
df['bb_up'], df['bb_down'] = bollinger_bands(df['Mid_Close'])
df['kc_up'], df['kc_down'] = keltner_channels(df['Mid_Close'], atr(df, lookback=10))

range_highs_lows_tups = [range_high_lows(df, idx) for idx in range(len(df))]
df['range_high'], df['range_low'] = [tup[0] for tup in range_highs_lows_tups], [tup[1] for tup in range_highs_lows_tups]

df['sar'] = psar(df)

df['chop14'] = chop(df)
df['chop36'] = chop(df, lookback=36)

df['vo'] = vo(df['Volume'])
df['vo_long'] = vo(df['Volume'], short_lookback=18, long_lookback=36)

df['len_avg'], df['len_std'] = bar_lengths(abs(df['Mid_High'] - df['Mid_Low']))
df['sar_len_avg'], df['sar_len_std'] = sar_lengths(df['Mid_Open'], df['sar'])

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df)

                     Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0     2021-04-01 10:25:00   0.75634   0.75666  0.75634    0.75659   0.75647   
1     2021-04-01 10:30:00   0.75656   0.75656  0.75621    0.75621   0.75668   
2     2021-04-01 10:35:00   0.75626   0.75643  0.75626    0.75636   0.75639   
3     2021-04-01 10:40:00   0.75633   0.75639  0.75624    0.75624   0.75646   
4     2021-04-01 10:45:00   0.75621   0.75636  0.75621    0.75623   0.75633   
...                   ...       ...       ...      ...        ...       ...   
74455 2022-04-01 05:35:00   0.74750   0.74773  0.74747    0.74767   0.74765   
74456 2022-04-01 05:40:00   0.74768   0.74768  0.74740    0.74756   0.74780   
74457 2022-04-01 05:45:00   0.74756   0.74770  0.74754    0.74766   0.74770   
74458 2022-04-01 05:50:00   0.74764   0.74766  0.74734    0.74762   0.74780   
74459 2022-04-01 05:55:00   0.74761   0.74763  0.74726    0.74752   0.74775   

       Ask_High  Ask_Low  Ask_Close  Mid_Open  ... 

In [7]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [8]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'JPY' else pips_to_risk * 100

    if second == 'USD':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'JPY' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, use_macd_levels, use_keltner):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    trade = None
    limit_order = None

    for i in range(1, len(df)):
        curr_date = df.loc[df.index[i], 'Date']
        adx, atr, atr_sma, mid_low1, mid_high1 = df.loc[df.index[i - 1], ['adx', 'atr', 'atr_sma', 'Mid_Low', 'Mid_High']]
        sideways = adx < 20 and atr < atr_sma
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_mid_open, curr_mid_low = \
            df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_Low']]

        if limit_order is None and trade is None and sideways:
            if use_macd_levels:
                range_high, range_low = df.loc[df.index[i - 1], ['range_high', 'range_low']]

            elif use_keltner:
                range_high, range_low = df.loc[df.index[i - 1], ['kc_up', 'kc_down']]

            else:
                range_high, range_low = df.loc[df.index[i - 1], ['bb_up', 'bb_down']]

            range_low, range_high = min(range_low, mid_low1), max(range_high, mid_high1)
            
            if range_high > range_low:
                limit_order = {'buy': float(range_low), 'sell': float(range_high), 'range': abs(float(range_high) - float(range_low))}

        if not sideways:
            limit_order = None

        if trade is None and limit_order is not None:
            place_buy = curr_ask_low <= limit_order['buy']
            place_sell = curr_bid_high >= limit_order['sell']

            if place_buy:
                open_price = float(limit_order['buy'])
                stop_gain = open_price + limit_order['range'] / 2
                pips_gained = stop_gain - open_price
                curr_pips_to_risk = pips_gained / risk_reward_ratio
                pullback = open_price - curr_pips_to_risk

                stop_loss = round(pullback, 5)

                if stop_loss < open_price and stop_gain > open_price and spread <= curr_pips_to_risk * spread_cutoff:
                      n_units = get_n_units('buy', stop_loss, open_price, curr_bid_open, curr_mid_open, 'USD_CAD')

                      trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                      'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                      'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                      n_buys += 1

                      pips_risked.append(curr_pips_to_risk)
                      
                      limit_order = None

            elif place_sell:
                open_price = float(limit_order['sell'])
                stop_gain = open_price - limit_order['range'] / 2
                pips_gained = open_price - stop_gain
                curr_pips_to_risk = pips_gained / risk_reward_ratio
                pullback = open_price + curr_pips_to_risk

                stop_loss = round(pullback, 5)

                if stop_loss > open_price and stop_gain < open_price and spread <= curr_pips_to_risk * spread_cutoff:
                      n_units = get_n_units('sell', stop_loss, curr_ask_open, open_price, curr_mid_open, 'USD_CAD')

                      trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                              'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                              'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                      n_sells += 1

                      pips_risked.append(curr_pips_to_risk)

                      limit_order = None

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None    

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [9]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [0.5, 1.0, 1.5]
# risk_reward_ratio_vals = [1.5]
spread_cutoffs = [0.10, 0.15, 0.20, 0.25, 0.30]
# spread_cutoffs = [0.20]
use_macd_vals = [True, False]
use_keltner_vals = [True, False]
n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(use_macd_vals) * len(use_keltner_vals)
all_combos = []


# use_half_sar, vo_cutoff, use_candles

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs: 
        for use_macd_levels in use_macd_vals:
            for use_keltner in use_keltner_vals:
                all_combos.append((risk_reward_ratio, spread_val, use_macd_levels, use_keltner))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_use_macd_val = None
best_use_keltner_val = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, use_macd_levels, use_keltner in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, use_macd_levels, use_keltner)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    # print('Avg pips risked: ' + str(sum(pips_risked) / len(pips_risked)))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread_val': spread_val, 'use_macd_levels': use_macd_levels, 'use_keltner': use_keltner})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_use_macd_val = use_macd_levels
        best_use_keltner_val = use_keltner

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 60

-6544.982435000011
Num buys: 198
Num sells: 248
Num trades: 446
Num wins: 109
Num losses: 337
Win streak: 5
Loss streak: 21
Remaining runs: 59
Best reward so far: -6544.982435000011

-3553.9443050000045
Num buys: 175
Num sells: 220
Num trades: 395
Num wins: 151
Num losses: 244
Win streak: 6
Loss streak: 10
Remaining runs: 58
Best reward so far: -3553.9443050000045

-1534.4000569208845
Num buys: 166
Num sells: 184
Num trades: 350
Num wins: 207
Num losses: 143
Win streak: 13
Loss streak: 6
Remaining runs: 57
Best reward so far: -1534.4000569208845

-2366.690224999981
Num buys: 105
Num sells: 145
Num trades: 250
Num wins: 94
Num losses: 156
Win streak: 4
Loss streak: 11
Remaining runs: 56
Best reward so far: -1534.4000569208845

-349.0204750000071
Num buys: 37
Num sells: 43
Num trades: 80
Num wins: 36
Num losses: 44
Win streak: 4
Loss streak: 5
Remaining runs: 55
Best reward so far: -349.0204750000071

-895.5524906483561
Num buys: 150
Num sells: 188
Num trades: 338
Num wins:

In [10]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best use macd levels val: ' + str(best_use_macd_val))
print('Best use keltner val: ' + str(best_use_keltner_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 219.42142500000773
Best risk/reward ratio: 1.5
Best spread: 0.1
Best use macd levels val: True
Best use keltner val: True
-----------------------
Top results:
{'reward': -1534, 'ratio': 0.5, 'spread_val': 0.1, 'use_macd_levels': False, 'use_keltner': True}
{'reward': -349, 'ratio': 1.0, 'spread_val': 0.1, 'use_macd_levels': True, 'use_keltner': True}
{'reward': -895, 'ratio': 0.5, 'spread_val': 0.1, 'use_macd_levels': False, 'use_keltner': False}
{'reward': -346, 'ratio': 1.0, 'spread_val': 0.1, 'use_macd_levels': True, 'use_keltner': False}
{'reward': -421, 'ratio': 1.0, 'spread_val': 0.1, 'use_macd_levels': False, 'use_keltner': False}
{'reward': -1287, 'ratio': 1.5, 'spread_val': 0.2, 'use_macd_levels': False, 'use_keltner': True}
{'reward': -1225, 'ratio': 0.5, 'spread_val': 0.1, 'use_macd_levels': True, 'use_keltner': False}
{'reward': -932, 'ratio': 1.0, 'spread_val': 0.15, 'use_macd_levels': False, 'use_keltner': False}
{'rewa

In [10]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best use macd levels val: ' + str(best_use_macd_val))
print('Best use keltner val: ' + str(best_use_keltner_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 386.2279221210685
Best risk/reward ratio: 1.0
Best spread: 0.1
Best use macd levels val: False
Best use keltner val: True
-----------------------
Top results:
{'reward': -2100, 'ratio': 0.5, 'spread_val': 0.1, 'use_macd_levels': False, 'use_keltner': True}
{'reward': -2188, 'ratio': 0.5, 'spread_val': 0.1, 'use_macd_levels': True, 'use_keltner': False}
{'reward': -1622, 'ratio': 1.0, 'spread_val': 0.15, 'use_macd_levels': False, 'use_keltner': True}
{'reward': -2757, 'ratio': 1.0, 'spread_val': 0.15, 'use_macd_levels': True, 'use_keltner': False}
{'reward': -2483, 'ratio': 1.5, 'spread_val': 0.15, 'use_macd_levels': True, 'use_keltner': False}
{'reward': -1513, 'ratio': 1.0, 'spread_val': 0.1, 'use_macd_levels': True, 'use_keltner': True}
{'reward': -2655, 'ratio': 0.5, 'spread_val': 0.15, 'use_macd_levels': False, 'use_keltner': False}
{'reward': -962, 'ratio': 1.5, 'spread_val': 0.15, 'use_macd_levels': False, 'use_keltner': False}

In [13]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best use macd levels val: ' + str(best_use_macd_val))
print('Best use keltner val: ' + str(best_use_keltner_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 328.9372980356628
Best risk/reward ratio: 0.5
Best spread: 0.1
Best use macd levels val: False
Best use keltner val: False
-----------------------
Top results:
{'reward': -255, 'ratio': 1.5, 'spread_val': 0.1, 'use_macd_levels': False, 'use_keltner': False}
{'reward': 230, 'ratio': 1.0, 'spread_val': 0.1, 'use_macd_levels': False, 'use_keltner': False}
{'reward': -953, 'ratio': 0.5, 'spread_val': 0.1, 'use_macd_levels': False, 'use_keltner': True}
{'reward': -1481, 'ratio': 1.5, 'spread_val': 0.15, 'use_macd_levels': True, 'use_keltner': False}
{'reward': -60, 'ratio': 1.5, 'spread_val': 0.1, 'use_macd_levels': False, 'use_keltner': True}
{'reward': -910, 'ratio': 1.0, 'spread_val': 0.15, 'use_macd_levels': False, 'use_keltner': False}
{'reward': 328, 'ratio': 0.5, 'spread_val': 0.1, 'use_macd_levels': False, 'use_keltner': False}
{'reward': 177, 'ratio': 1.5, 'spread_val': 0.15, 'use_macd_levels': False, 'use_keltner': False}
{'rewa

In [10]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best use macd levels val: ' + str(best_use_macd_val))
print('Best use keltner val: ' + str(best_use_keltner_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: -62.566476632554895
Best risk/reward ratio: 1.5
Best spread: 0.1
Best use macd levels val: False
Best use keltner val: True
-----------------------
Top results:
{'reward': -1883, 'ratio': 0.5, 'spread_val': 0.1, 'use_macd_levels': False, 'use_keltner': True}
{'reward': -2008, 'ratio': 1.0, 'spread_val': 0.1, 'use_macd_levels': True, 'use_keltner': False}
{'reward': -2429, 'ratio': 1.0, 'spread_val': 0.15, 'use_macd_levels': False, 'use_keltner': True}
{'reward': -1839, 'ratio': 0.5, 'spread_val': 0.1, 'use_macd_levels': True, 'use_keltner': False}
{'reward': -570, 'ratio': 1.5, 'spread_val': 0.1, 'use_macd_levels': True, 'use_keltner': True}
{'reward': -484, 'ratio': 1.5, 'spread_val': 0.15, 'use_macd_levels': False, 'use_keltner': True}
{'reward': -1582, 'ratio': 1.5, 'spread_val': 0.15, 'use_macd_levels': False, 'use_keltner': False}
{'reward': -62, 'ratio': 1.5, 'spread_val': 0.1, 'use_macd_levels': False, 'use_keltner': True}
{'r